In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.getenv('MLFLOW_TRACKING_URI')

'http://127.0.0.1:8080'

In [3]:
import sys
print(sys.version)

3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]


In [4]:
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [5]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

In [6]:
data.shape

(4898, 12)

In [7]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [8]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [9]:
train.shape

(3673, 12)

In [10]:
test.shape

(1225, 12)

In [11]:
train_x = train.drop(["quality"], axis=1)


In [12]:
train_y = train["quality"].values.ravel()
train_y

array([6, 6, 5, ..., 6, 6, 8])

In [13]:
test_x = test.drop(["quality"], axis=1)
test_y = test["quality"].values.ravel()

In [14]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

# Define the input shape and dtype
input_shape = (-1, train_x.shape[1])  # -1 for variable batch size
input_dtype = np.dtype(np.float32)

# Create the input schema using TensorSpec
input_schema = Schema([
    TensorSpec(type=input_dtype, shape=input_shape, name=None)
])
output_schema = Schema([
    TensorSpec(type=np.dtype(np.float32), shape=(-1, 1), name=None)
])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [16]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [17]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [18]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [21]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/06/09 00:28:14 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 14s 331ms/step - loss: 33.8901 - root_mean_squared_error: 5.8215
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.9713 - root_mean_squared_error: 5.7420   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 32.9071 - root_mean_squared_error: 5.7364 - val_loss: 32.2029 - val_root_mean_squared_error: 5.6748

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 31.9137 - root_mean_squared_error: 5.6492
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.8752 - root_mean_squared_error: 5.6458 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.8289 - root_mean_squared_error: 5.6417 - val_loss: 30.9933 - val_root_mean_squared_error: 5.5672

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 30.9382 - root_mean_squared_error: 5.5622
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.5645 - root_mean_squared_error: 5.5